<a href="https://colab.research.google.com/github/pyrated03/IISc-Coding-Test/blob/main/PACS_Domain_Generalization(IISc_Coding_Test)_Adarsh_Subramanian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Clearing GPU memory before training

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import gc
# del variables
gc.collect()
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)
import os
os.environ['CUDA_VISIBLE_DEVICES']='2, 3'

## MOUNTING DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


INSTALLING RESNET_PYTORCH LIBRARY

In [ ]:
!pip3 install resnet_pytorch

### UNZIPPING THR GIVEN PACS.zip DATA FILE TO USE FOR TRAINING. 
THE BELOW LINE HAS BEEN COMMENTED AS THE FILE HAS BEEN UNZIPPED AND NEED NOT BE DONE AGAIN.

In [ ]:
# !unzip gdrive/My\ Drive/IISc\ Coding\ Test/PACS.zip -d gdrive/My\ Drive/IISc\ Coding\ Test

IMPORTING NECESSARY LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import PIL
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision.utils import make_grid

import torchvision.transforms as T
import torch
import torch.nn as nn
import torchvision
from torchvision.utils import make_grid
from torchvision.utils import save_image    
from IPython.display import Image
import random


import torch.optim as optim
import torch.nn.functional as F
from torchvision import *
import time
import copy
from torch.optim import lr_scheduler


DEFINING NECESSARY TRANSFORMS

In [ ]:
# add transforms to the data
# some of the transformations have been commented out due to GPU constraints
data_transforms = {
    'train': transforms.Compose([
        # transforms.RandomRotation(45),
        # transforms.RandomResizedCrop(224),
        # transforms.RandomHorizontalFlip(),
        transforms.Resize((32)),#attention 
        # transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], 
                            # [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize((32))
    ]),
    'test': transforms.Compose([
        transforms.Resize((32))
    ])
}

### **DATALOADER**

In [ ]:
class MyDataset(Dataset):

    def __init__(self, train = True, transforms = None):
        
        self.transforms = transforms
        if train:
            path_p = "gdrive/My Drive/IISc Coding Test/kfold/photo"
            path_a = "gdrive/My Drive/IISc Coding Test/kfold/art_painting"
            path_c = "gdrive/My Drive/IISc Coding Test/kfold/cartoon"

            path_labels = os.listdir(path_p)
            self.label_dict = {}
            i = 0
            for l in path_labels:
                self.label_dict[l] = i
                i+=1

            # print(path_labels)
            self.images_label_dict = {}

            for label in path_labels:
                path_1 = os.listdir(path_p+"/"+label)
                # count+=len(path_1)
                path_2 = os.listdir(path_a+"/"+label)
                # count+=len(path_2)
                path_3 = os.listdir(path_c+"/"+label)
                # count+=len(path_3)

                for path_img in path_1:
                    path_img = path_p+"/"+label+"/"+path_img
                    self.images_label_dict[path_img] = label 

                for path_img in path_2:
                    path_img = path_a+"/"+label+"/"+path_img
                    self.images_label_dict[path_img] = label

                for path_img in path_3:
                    path_img = path_c+"/"+label+"/"+path_img
                    self.images_label_dict[path_img] = label

        else:
            path_test = "gdrive/My Drive/IISc Coding Test/kfold/sketch"
            path_labels = os.listdir(path_test)
            self.label_dict = {}
            i = 0
            for l in path_labels:
                self.label_dict[l] = i
                i+=1

            # print(path_labels)
            self.images_label_dict = {}

            for label in path_labels:
                path_1 = os.listdir(path_test+"/"+label)

                for path_img in path_1:
                    path_img = path_test+"/"+label+"/"+path_img
                    self.images_label_dict[path_img] = label 
       
    def __getitem__(self, index):
        keys = list(self.images_label_dict.keys())
        values = list(self.images_label_dict.values())
        image = read_image(keys[index])
        label = self.label_dict[values[index]]

        # sample = {'x':image, 'y':label}
        # return sample
        if self.transforms:
          image = self.transforms(image)
          
        return image,label

    def __len__(self):
        return len(self.images_label_dict)

LOADING TRAIN(SOURCE) DATA AND SPLITTING 10% OF IT FOR VALIDATION

In [ ]:
dset = MyDataset(train=True, transforms = data_transforms['train'])
batch_size = 4
validation_split = 0.1 #10% of source domain kept as validation set
shuffle_dataset = True
random_seed= 42

from torch.utils.data.sampler import SubsetRandomSampler

dataset_size = len(dset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_ds = DataLoader(dset, batch_size=batch_size, sampler=train_sampler)
dset = MyDataset(train=True, transforms = data_transforms['valid'])
validation_ds = DataLoader(dset, batch_size=batch_size, sampler=valid_sampler)


LOADING TEST(TARGET) DATA

In [ ]:
test_dset = MyDataset(train = False, transforms = data_transforms['test'])
batch_size = 4
random_seed= 42

test_ds = DataLoader(test_dset, batch_size=batch_size)

In [ ]:
dataset_sizes = {'train' : len(train_ds)*batch_size, 'valid' : len(validation_ds)*batch_size}
print(len(train_ds))
print(len(validation_ds))
print(len(test_ds))


1364
152
983


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

### **DEFINING RESNET-18 AS THE FEATURE EXTRACTOR**

In [ ]:
model = models.resnet18(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### **DEFINING THE FINAL LAYER FOR CLASSIFICATION**

In [ ]:
model.fc = nn.Linear(512, 7)
model = model.to(device)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### **DEFINING A FUNCTION TO TRAIN THE MODEL, AND RETURN THE LOSS COMPONENTS**

In [ ]:
def train_model(model, criteria, optimizer, scheduler,device, num_epochs=25,lambda1=0.33,lambda2=0.33,lambda3=0.33):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    c_no = 0
    bol = True
    loss_a_array = []
    loss_b_array = []
    loss_c_array = []
    for epoch in range(1, num_epochs+1):
        print('Epoch {}/{}'.format(epoch, num_epochs ))
        print('-' * 10)
        iter = 0
        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            loss = torch.zeros(4,7).to(device)
            # Iterate over data.
            for inputs, labels in train_ds:
                if iter%100 == 0:
                  print('Iterations :', iter)

                iter+=1
                # print("Inside Train")
                inputs = inputs.float().to(device)
                labels = labels.to(device)
                print(inputs.shape)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                try:
                    bol = True
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs_train = model(inputs)
                        _, preds = torch.max(outputs_train, 1)
                        loss_a = criteria(outputs_train, labels)
                        # print("loss a ", loss_a.shape)
                        loss_b = torch.zeros(7,4).to(device)
                        # print("loss b ", loss_b.shape)
                        loss_c = torch.zeros(4,7).to(device)

                        # print("loss c ", loss_b.shape)
                        
                        for inputs_test, labels_test in test_ds:
                            # print("Inside Test")
                            inputs_test = inputs_test.float().to(device)
                            # print("Input_Test shape ",inputs_test.shape)
                            outputs_test = model(inputs_test)
                            # print("Output_Test shape ",outputs_test.shape)
                            _,sigma = torch.max(outputs_test,1) #Add a 1?
                            ypred = torch.argmax(outputs_test,1)
                            cmp = torch.ones(len(outputs_test)) * 0.95
                            cmp = cmp.to(device)
                            cmp = torch.ge(sigma, cmp)
                            loss_b+=torch.mul(torch.mul(sigma, criteria(outputs_test, ypred)),cmp)
                            loss_c+= -torch.mul((outputs_test), torch.log(outputs_test))

                        loss = lambda1 * loss_a + lambda2 * loss_b + lambda3 * loss_c

                        # taking mean of each loss components 
                        loss_a_mean = torch.mean(loss_a)
                        loss_b_mean = torch.mean(loss_b)
                        loss_c_mean = torch.mean(loss_c)
                        loss_a_array.append(loss_a_mean)
                        loss_b_array.append(loss_b_mean)
                        loss_c_array.append(loss_c_mean)


                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                except:
                    c_no = c_no + 1
                    bol = False
                    loss = torch.mul(loss, 0)
                # statistics
                if bol:
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            print("No of catches: ",c_no)
            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model,loss_a_array,loss_b_array,loss_c_array

In [ ]:
criteria = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
# Number of epochs
eps=5


## **TRAINING THE MODEL**

In [ ]:
model,loss_a_array,loss_b_array,loss_c_array = train_model(model, criteria, optimizer, scheduler,device, eps)

### **PLOTTING THE LOSS COMPONENTS RETURNED FROM TRAINING**

In [ ]:
plt.plot(loss_a_array,color = 'r', label = 'loss_a')
plt.plot(loss_b_array,color = 'b', label = 'loss_b')
plt.plot(loss_c_array,color = 'g', label = 'loss_c')
plt.xlabel("Training Progress --->")
plt.ylabel("Loss")
plt.legend()